In [1]:
import pandas as pd
# Functions for ABC classification and inventory metrics go here

In [2]:
# Load cleaned data and prepare inventory summary
import pandas as pd
from pathlib import Path

CLEANED_DIR = Path('d:/slooze_challenge/Slooze_Analysis/data/cleaned')
OUT_DIR = Path('d:/slooze_challenge/Slooze_Analysis/results')
OUT_DIR.mkdir(parents=True, exist_ok=True)

sales = pd.read_csv(CLEANED_DIR / 'sales_cleaned.csv', parse_dates=['date'], low_memory=False)
prices_path = CLEANED_DIR / 'prices_cleaned.csv'
prices = pd.read_csv(prices_path, low_memory=False) if prices_path.exists() else None

# Create a simple SKU summary (quantity sold, revenue). We'll merge costs if available.
sku_summary = sales.groupby('sku').agg(
    total_qty_sold=('quantity','sum'),
    total_revenue=('revenue','sum')
).reset_index().sort_values('total_revenue', ascending=False)

if prices is not None:
    # try to detect sku and cost columns
    sku_col = next((c for c in prices.columns if 'sku' in c.lower() or 'item' in c.lower()), None)
    cost_col = next((c for c in prices.columns if 'cost' in c.lower() or 'price' in c.lower()), None)
    if sku_col and cost_col:
        prices = prices.rename(columns={sku_col: 'sku', cost_col: 'unit_cost'})
        sku_summary = sku_summary.merge(prices[['sku','unit_cost']], on='sku', how='left')
    else:
        sku_summary['unit_cost'] = pd.NA
else:
    sku_summary['unit_cost'] = pd.NA

# Estimate inventory value (roughly)
sku_summary['inventory_value_est'] = sku_summary['unit_cost'] * sku_summary['total_qty_sold']
sku_summary.to_csv(OUT_DIR / 'sku_inventory_summary.csv', index=False)
print('Saved SKU inventory summary to results/sku_inventory_summary.csv')


Saved SKU inventory summary to results/sku_inventory_summary.csv


In [3]:
# ABC classification: divide products into A/B/C by cumulative revenue share
sku_summary = sku_summary.sort_values('total_revenue', ascending=False).reset_index(drop=True)
sku_summary['cum_revenue'] = sku_summary['total_revenue'].cumsum()
sku_summary['cum_revenue_pct'] = 100 * sku_summary['cum_revenue'] / sku_summary['total_revenue'].sum()

def abc_class(pct):
    if pct <= 80:
        return 'A'
    elif pct <= 95:
        return 'B'
    else:
        return 'C'

sku_summary['ABC'] = sku_summary['cum_revenue_pct'].apply(abc_class)
sku_summary.to_csv(OUT_DIR / 'sku_abc.csv', index=False)
print('Saved ABC classification to results/sku_abc.csv')
print('Counts by class:')
print(sku_summary['ABC'].value_counts())


Saved ABC classification to results/sku_abc.csv
Counts by class:
ABC
A    71626
B    51345
C    47160
Name: count, dtype: int64
